In [34]:
import pandas as pd
import numpy as np

In [35]:
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
import pickle

Load model and scalers here:

In [36]:
LR2 = pickle.load(open('models/LR2.sav', 'rb'))
scaler_years = pickle.load(open('scaler/YearsCode.sav', 'rb'))
scaler_comp = pickle.load(open('scaler/ConvertedCompYearly.sav', 'rb'))

Load dataset here:

In [49]:
data = pd.read_csv('csv-data/clean.csv').drop(['Unnamed: 0', 'Country'], axis=1)

In [50]:
def splitter(x: str):
    return x.split('(')[0].rstrip()

In [39]:
def encode_multichoices(dataseries, column, i=0):
    df = pd.DataFrame(dataseries[column])
    df = df[column].str.get_dummies(sep=';')
    df = df.drop(['Other (Please specify):'], axis=1, errors='ignore')
    df = df.drop(['Other (please specify):'], axis=1, errors='ignore')
    df.columns = list(map(splitter, df.columns))  
    return df

In [61]:
cats = pd.DataFrame([])
i = 0
for col in data.select_dtypes(include=object).columns:
    encoded = encode_multichoices(data, col)
    encoded = encoded.add_prefix(f'{i}_')
    cats = pd.concat([cats, encoded], axis=1)
    i += 1

data['YearsCode'] = scaler_years.transform(pd.DataFrame(data['YearsCode']))
data['ConvertedCompYearly'] = scaler_comp.transform(pd.DataFrame(data['ConvertedCompYearly']))

# concat with numerical
treated = pd.concat([data.select_dtypes(include=np.number), cats], axis=1)

Run data trough model:

In [66]:
predictions = pd.DataFrame(LR2.predict(treated))
predictions

,0
0,"Developer, full-stack"
1,"Developer, back-end"
2,Engineering manager
3,"Developer, full-stack"
4,"Developer, full-stack"
...,...
14298,Engineering manager
14299,"Developer, full-stack"
14300,"Developer, front-end"
14301,"Developer, back-end"
